# Forward reasoning in First Order Logic
-Arjun A.S
1BM18CS019

In [1]:

#Program for FORWARD REASONING

import re #Regular expression module

def isVariable(x):
    return len(x) == 1 and x.islower() and x.isalpha()

def getAttributes(string):
    expr = '\([^)]+\)'
    matches = re.findall(expr, string)
    return matches

def getPredicates(string):
    expr = '([a-z~]+)\([^&|]+\)'
    return re.findall(expr, string)

In [2]:
class Fact:    
    def __init__(self, expression):
        self.expression = expression
        predicate, params = self.splitExpression(expression)
        self.predicate = predicate
        self.params = params
        self.result = any(self.getConstants())
        
    def splitExpression(self, expression):
        predicate = getPredicates(expression)[0]
        params = getAttributes(expression)[0].strip('()').split(',')
        return [predicate, params]
    
    def getConstants(self):
        return [None if isVariable(c) else c for c in self.params]
    
    def getResult(self):
        return self.result
    
    def getVariables(self):
        return [v if isVariable(v) else None for v in self.params]

In [3]:
class Implication:
    def __init__(self, expression):
        self.expression = expression
        spl = expression.split('=>')
        self.lhs = [Fact(f) for f in spl[0].split('&')]
        self.rhs = Fact(spl[1])
        
    def evaluation_implication(self, facts):
        constants = {}
        new_lhs = []   #facts that can prove the RHS
        for fact in facts:
            for clause in self.lhs:
                if clause.predicate == fact.predicate:      #comparision btw Known facts and the implication
                    for i, v in enumerate(clause.getVariables()):
                        if v:
                            constants[v] = fact.getConstants()[i]
                    new_lhs.append(fact)
        predicate= getPredicates(self.rhs.expression)[0] 
        attributes=str(getAttributes(self.rhs.expression)[0])
        for key in constants:
            if constants[key]:
                attributes = attributes.replace(key, constants[key])
        expr = f'{predicate}{attributes}'
        return Fact(expr) if len(new_lhs) and all([f.getResult() for f in new_lhs]) else None

In [4]:
class KnowledgeBase:
    def __init__(self):
        self.facts = set()
        self.implications = set()
            
    def tell(self, e): #function to enter facts into the knowledgebase
        if '=>' in e: 
            self.implications.add(Implication(e))
        else:
            self.facts.add(Fact(e))
        for i in self.implications:
            res = i.evaluation_implication(self.facts)
            if res:
                self.facts.add(res)

    def ask(self, exp):  #function to query the Knowldege base
        facts = set([f.expression for f in self.facts])
        i = 1
        print(f'Querying {exp}:')
        for f in facts:
            if Fact(f).predicate == Fact(exp).predicate:
                print(f'\t{i}. {f}')
                i += 1
    
    def display(self):
        print("All facts in Knowledge Base:")
        for i, f in enumerate(set([f.expression for f in self.facts])):
            print(f'\t{i+1}. {f}')

In [5]:
def main():
    kb = KnowledgeBase()
    print("Enter clauses for KB: (Enter 'exit' to Stop telling the KnowledgeBase)")
    while True:
        clause = input()
        if(clause == 'exit'):
            break
        kb.tell(clause)
    query = input("Enter Query:")
    kb.ask(query)
    kb.display()

In [9]:
if __name__ =="__main__":
    main()  

Enter clauses for KB: (Enter 'exit' to Stop telling the KnowledgeBase)
food(x)=>likes(Rani,x)
food(Peanut)
~food(Mug)
exit
Enter Query:likes(Rani,Peanut)
Querying likes(Rani,Peanut):
	1. likes(Rani,Peanut)
All facts in Knowledge Base:
	1. ~food(Mug)
	2. likes(Rani,Peanut)
	3. food(Peanut)


In [7]:
#food(x)=>likes(Rani,x) can be written as 
# ~[food(x)]V[likes(Rani,x)]     in horn clause form , as every implication is a definite clause

In [10]:
if __name__ =="__main__":
    main()  

Enter clauses for KB: (Enter 'exit' to Stop telling the KnowledgeBase)
food(Peanut)
~food(Mug)
food(y)=>likes(Ram,y)
exit
Enter Query:likes(Rani,y)
Querying likes(Rani,y):
	1. likes(Ram,Peanut)
All facts in Knowledge Base:
	1. food(Peanut)
	2. ~food(Mug)
	3. likes(Ram,Peanut)
